In [2]:
import pandas as pd
import numpy as np

Data preprocessing | filtering | notnull |

In [3]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [5]:
movies = movies.merge(credits, on='title')

In [ ]:
movies.head()

In [6]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [ ]:
movies.isnull().sum()

In [9]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

Stringifying the data from the json -> Values

In [16]:
import ast
def convert(obj):
  list = []
  for i in ast.literal_eval(obj):
    list.append(i['name'])
  return list[0:3]

In [19]:
def find(obj):
  list = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      list.append(i['name'])
  return list

In [14]:
movies['genres'] = movies['genres'].apply(convert)

In [15]:
movies['keywords'] = movies['keywords'].apply(convert)

In [17]:
movies['cast'] = movies['cast'].apply(convert)

In [20]:
movies['crew'] = movies['crew'].apply(find)

In [27]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [24]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

vaibhav kokare -> vaibhavkokare = easy when analyzing

In [26]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])

In [28]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [29]:
db = movies[['movie_id', 'title', 'tags']]

In [31]:
db['tags'] = db['tags'].apply(lambda x:" ".join(x))

<ipython-input-31-adc71f8a3649>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['tags'] = db['tags'].apply(lambda x:" ".join(x))


In [33]:
db.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [64]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=5000, stop_words='english')

In [69]:
vectors = vectorizer.fit_transform(db['tags']).toarray()
vectors.shape

(1831, 5000)

Remove multi verb froms -> trust, trusted -> trust
check first ten stem then again check the words

In [ ]:
for i in vectorizer.get_feature_names_out():
  print(i)

In [56]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [59]:
def stemming(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [60]:
stemming('trusted')

'trust'

In [63]:
db['tags'] = db['tags'].apply(stemming)

<ipython-input-63-517fb3b9e1b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['tags'] = db['tags'].apply(stemming)
